## Install Dependencies

In [1]:
!pip install accelerate==0.21.0 \
  bitsandbytes==0.40.2 \
  peft==0.5.0 \
  transformers==4.34.0 \
  sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: accelerate
    

In [2]:
!pip install peft

In [3]:
!pip install streamlit
!npm install localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 58.2 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.0 MB/s eta 0:00:00
m#################.] - reify:debug: http fetch GET 200 https://registry.npmjs.o
added 22 packages in 2s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.1.0 -> 10.5.1
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.5.1
npm notice Run npm install -g npm@10.5.1 to update!
npm notice 


## Data Reader

In [4]:
%%writefile data_reader.py

import pandas as pd


df_ste = pd.read_excel("/kaggle/input/tenderhack-msk/TenderHack_msk.xlsx", sheet_name="СТЕ")
df_char = pd.read_excel("/kaggle/input/tenderhack-msk/TenderHack_msk.xlsx", sheet_name="Характеристики")

df_ste["Название СТЕ"] = df_ste["Название СТЕ"].str.lower()
df_ste["Наименование конечной категории Портала"] = df_ste["Наименование конечной категории Портала"].str.lower()

unique_targets = df_ste["Наименование конечной категории Портала"].unique()
target2id = {target: idx for idx, target in enumerate(unique_targets)}
id2target = {idx: target for idx, target in enumerate(unique_targets)}

test_top = pd.read_csv("/kaggle/input/top-15-characteristic-for-category/top_15_characteristic_for_category.csv")
test_top["Наименование конечной категории Портала"] = test_top["Наименование конечной категории Портала"].str.lower()
category_dict = test_top.groupby('Наименование конечной категории Портала')['Название характеристики'].apply(list).to_dict()

Writing data_reader.py


## Ozon Parser

In [5]:
%%writefile ozon_parser.py

import re
from urllib.parse import quote
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup


def get_encode_url_from_source_text(text: str) -> str:
    """ Кодирование запроса в ссылке """
    encoded_text = quote(text, safe='')
    url = f"https://www.ozon.ru/search/?text={encoded_text}&from_global=true"
    return url


def get_soup_from_url(url: str):
    r = Request(url)
    html = urlopen(r).read()
    soup = BeautifulSoup(html, features="html.parser")
    return soup


def get_all_product_from_url(soup) -> list[str]:
    """ Получить все ссылки на товары на странице запроса """
    product_urls = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and '/product/' in href:
            product_urls.append(href)
    return product_urls


def get_text_from_url(url: str) -> str:
    """ Получение текста со страницы """
    soup = get_soup_from_url(url)
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text


def clean_string(text, trash_words):
    """ Чистка текста [Deprecated] """
    text = text.split("\nОтзывы о товаре", 1)[0]
    text = text.split("\nХарактеристики", 1)[1]
    for word in trash_words:
        text = re.sub(r'\n' + word + r'.*?(?=\n|$)', '', text)
    return text


def get_clean_from_roma(url):
    """ Чистка текста """
    text = get_text_from_url(url)
    text = text.split("Отзывы о товаре", 1)[0]
    trimmed_string = text.split("Подборки товаров")[0]
    if trimmed_string.endswith(" "):
        trimmed_string = trimmed_string[:-1]
    text = trimmed_string.split("Характеристики")[1]
    return text


def get_products_urls_from_query(query):
    """ Получение ссылок на товары в маркетплейсе Ozon """
    OZON_SRC_URL = "https://www.ozon.ru"
    encoded_query = get_encode_url_from_source_text(query)
    soup = get_soup_from_url(encoded_query)
    urls = [OZON_SRC_URL + url for url in get_all_product_from_url(soup)]
    return urls


def get_characteristics_from_query(query):
    urls = get_products_urls_from_query(query)
    if len(urls) == 0:
        return None
    else:
        text_characteristics = get_clean_from_roma(urls[0])
    return text_characteristics


Writing ozon_parser.py


## cointegrated/rubert-tiny

In [6]:
%%writefile category_model.py

from data_reader import target2id, id2target

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


class CustomDataset(Dataset):
    def __init__(self, texts, targets, target2id, tokenizer, max_len=512):
        self.texts = texts
        self.targets = targets
        self.target2id = target2id
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.target2id[self.targets[idx]]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }



class BertClassifier:
    def __init__(self, model_path, tokenizer_path, n_classes, epochs, model_save_path='bert.pt'):
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)

    def preparation(self, X_train, y_train, X_valid, y_valid):
        self.train_set = CustomDataset(X_train, y_train, target2id, self.tokenizer)
        self.valid_set = CustomDataset(X_valid, y_valid, target2id, self.tokenizer)

        self.train_loader = DataLoader(self.train_set, batch_size=32, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=32, shuffle=True)

        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)

    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)

            losses.append(loss.item())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.optimizer.zero_grad()

        train_acc = correct_predictions.double() / len(self.train_set)
        train_loss = np.mean(losses)
        return train_acc, train_loss

    def eval(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())

        val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss

    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)

    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }

        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)

        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )

        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]
        return prediction


TOKENIZER_PATH = 'cointegrated/rubert-tiny'
MODEL_PATH = 'cointegrated/rubert-tiny'
MAX_EPOCHS = 10
MODEL_SAVE_PATH = 'bert.pt'
    
model = BertClassifier(MODEL_PATH, TOKENIZER_PATH, len(target2id), MAX_EPOCHS, MODEL_SAVE_PATH)
model.model = torch.load("/kaggle/input/rubert-classification/pytorch/v1/1/bert (3).pt", map_location=torch.device('cuda'))

Writing category_model.py


## saiga_mistral_7b_lora

In [7]:
%%writefile llm_model.py

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Вы - эксперт по анализу товаров и технических спецификаций. Ваша задача - извлечь характеристики из описания товара, представленного в виде текста. Каждая характеристика имеет свое название и соответствующее значение."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


def generate_ste(query, text_characteristics, model, tokenizer, generation_config, top_characteristics):
    inputs = [
        f"""
        Вы решили автоматизировать процесс создания JSON-файлов с характеристиками товаров на основе предоставленных текстовых данных. 
        Вам нужно создать JSON-объект, содержащий характеристики товара, извлеченные из текста. Обязательные характеристики: наименование, категория, бренд, модель. Остальные нужно вычленить из текста.
        Необходимо сделать результат на русском языке, кроме наименований. Входной текст: {text_characteristics}
        """
#         f"""
#         Вы решили автоматизировать процесс создания JSON-файлов с характеристиками товаров на основе предоставленных текстовых данных. 
#         Вам нужно создать JSON-объект, содержащий характеристики товара, извлеченные из текста.
#         Обязательные характеристики: наименование, категория, бренд, модель. 
#         Также нужно добавить остальные характеристики товара из текста.
#         Выход модели должен быть исключительно на русском языке, кроме наименований. Если обязательная характеристика не встретилась, запиши null.
#         Входной текст: {query}. {text_characteristics}"""
#         f"""
#         Создайте JSON-объект с характеристиками товара на основе предоставленных данных.
#         Обязательные характеристики: наименование, категория, бренд, модель, {top_characteristics}.
#         Также добавь неограниченное количество дополнительных характеристик, которые найдешь во входных данных.
#         Все характеристики должны быть на русском языке, за исключением наименования.
#         Входные данные: {query}. {text_characteristics}
#         """
#       f"""
#       Создайте JSON-объект с характеристиками товара на основе предоставленных данных. Обязательные характеристики включают наименование, категорию, бренд и модель товара, а также {top_characteristics}. Дополнительно добавьте неограниченное количество характеристик, которые можно найти во входных данных. Все характеристики, кроме наименования, должны быть представлены на русском языке.
#       Входные данные: {query}. {text_characteristics}
#       Убедитесь, что вы также включаете дополнительные характеристики, если они есть, даже если они не указаны явно в обязательных характеристиках.
#       """
    ]
    outputs = []
    for inp in inputs:
        conversation = Conversation()
        conversation.add_user_message(inp)
        prompt = conversation.get_prompt(tokenizer)

        output = generate(model, tokenizer, prompt, generation_config)
        outputs.append(output)
        print(prompt)
        print(output)
        print()
        print("==============================")
        print()
    return outputs


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Writing llm_model.py


## StreamLit

In [8]:
%%writefile app.py

import json
import pandas as pd
import streamlit as st
from ozon_parser import get_characteristics_from_query


def read_data():
#     st.write("""Считывание исходных данных...""")
    from data_reader import (
        df_ste,
        df_char,
        target2id,
        id2target,
        category_dict
    )
#     st.write("""Данные считаны.""")
    return df_ste, df_char, target2id, id2target, category_dict


def bert_model_init():
#     st.write("""Начало инициализации BERT-модели...""")
    from category_model import (
        model
    )
#     st.write("""BERT-Модель загружена.""")
    return model


def llm_model_init():
#     st.write("""Начало инициализации LLM-модели...""")
    from llm_model import (
        generate_ste,
        model, 
        tokenizer, 
        generation_config
    )
#     st.write("""LLM-Модель загружена.""")
    return generate_ste, model, tokenizer, generation_config


def input_text():
    query = st.text_input('Ввод наименования товара')
    if query is not None:
        characteristics = get_characteristics_from_query(query)
        if characteristics is None:
            st.write("""Некорректный ввод. Попробуйте ещё раз.""")
#     st.write("""Результат парсинга:""")
#     st.write(characteristics)
        return query, characteristics


def get_category_and_top_characteristics(query, bert_model, id2target, category_dict):
    category_id = bert_model.predict(query)
    category_name = id2target[category_id]
    top_characteristics = category_dict[category_name]
    return category_name, top_characteristics


def upload():
    df_ste, df_char, target2id, id2target, category_dict = read_data()
    
    bert_model = bert_model_init()
    generate_ste, llm_model, tokenizer, generation_config = llm_model_init()
    
    st.title("ГЕНЕРАТОР ХАРАКТЕРИСТИК СТЕ")
    st.write("""Веб-сайт команды GibData для генерации характеристик товаров.""")
    
    query, characteristics = input_text()
    
    category, top_characteristics = get_category_and_top_characteristics(query, bert_model, id2target, category_dict)
#     st.write(f"""Категория: {category}""")
#     st.write(f"""Лучшие характеристики: {top_characteristics}""")
    
    st.write("""Генерация характеристик товара...""")
    outputs = generate_ste(query, characteristics, llm_model, tokenizer, generation_config, top_characteristics)
    st.write("""\nРезультат:""")
    st.write(outputs)
    
    try:
        json_data = json.loads(outputs[0])
        df = pd.DataFrame.from_dict(json_data, orient='index').transpose()
        edited_df = st.data_editor(df)
    except:
        pass


if __name__ == "__main__":
    upload()


Writing app.py


## Running App

In [9]:
!curl ipv4.icanhazip.com

35.202.163.148


In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://gold-pillows-stick.loca.lt
